In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import NaiveBayesClassifier
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer
import re
# using the variable sw to hold all stopwords that are in English
sw = stopwords.words('indonesian')

In [2]:
ds = pd.read_csv(r"D:\UI\ANMEDSOS\Praktikum\KA\Data1.csv")

In [3]:
# Sebelum Menambahkan 
ds.head()

,label,id,waktu,text
0,-1.0,1.390000e+18,Fri Apr 23 12:38:07 +0000 2021,sch! katanya nih tahun ajaran baru nanti sekol...
1,-1.0,1.390000e+18,Fri Apr 23 11:40:07 +0000 2021,jujur aja gamau kuliah offline di kelas. palin...
2,-1.0,1.390000e+18,Fri Apr 23 11:00:30 +0000 2021,@collegemenfess Jangan terlalu semangat masuk ...
3,-1.0,1.390000e+18,Fri Apr 23 09:16:45 +0000 2021,@fullshan_ Iya asli abis kelas tuh kyk cape ba...
4,-1.0,1.390000e+18,Fri Apr 23 08:19:58 +0000 2021,sch! kalo kelas 12 itu butuh tambahan les offl...


In [4]:
ds['text'] = ds['text'].str.lower()
ds.dropna(subset=['label'], inplace=True)
ds

,label,id,waktu,text
0,-1.0,1.390000e+18,Fri Apr 23 12:38:07 +0000 2021,sch! katanya nih tahun ajaran baru nanti sekol...
1,-1.0,1.390000e+18,Fri Apr 23 11:40:07 +0000 2021,jujur aja gamau kuliah offline di kelas. palin...
2,-1.0,1.390000e+18,Fri Apr 23 11:00:30 +0000 2021,@collegemenfess jangan terlalu semangat masuk ...
3,-1.0,1.390000e+18,Fri Apr 23 09:16:45 +0000 2021,@fullshan_ iya asli abis kelas tuh kyk cape ba...
4,-1.0,1.390000e+18,Fri Apr 23 08:19:58 +0000 2021,sch! kalo kelas 12 itu butuh tambahan les offl...
...,...,...,...,...
2947,1.0,1.380000e+18,Sun Apr 18 09:33:59 +0000 2021,@mrkyouns @subschfess kemarin sih kepala dinas...
2948,1.0,1.390000e+18,Fri Apr 23 02:28:37 +0000 2021,"@puppysgyu ih gemush warnanya・・・ｻ anyway, enak..."
2949,1.0,1.380000e+18,Wed Apr 21 00:52:50 +0000 2021,pengen sekolah offline ・・・ｭ
2950,1.0,1.380000e+18,Tue Apr 20 13:05:15 +0000 2021,@otelapan gapapa sampe jumat aja aku dan abin ...


In [5]:
import time
from datetime import timedelta
import pandas as pd
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re

start_time = time.time()

In [6]:
ds['text'] = ds.text.astype(str)
swfactory = StopWordRemoverFactory()
stopword = swfactory.create_stop_word_remover()
stfactory = StemmerFactory()
stemmer = stfactory.create_stemmer()
tok = WordPunctTokenizer()

removehtml = r'http\S+'
removemention = r'@\S+'
removewww = r'www.\S+'
removepic = r'pic\S+'

In [7]:
def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    soup1 = BeautifulSoup(souped, 'html.parser')
    souped1 = soup1.get_text()
    stripped = re.sub(removehtml,'',souped1)
    stripped = re.sub(removemention,'',stripped)
    stripped = re.sub(removewww, '', stripped)
    stripped = re.sub(removepic, '', stripped)
    letters_only = re.sub('[^a-zA-Z]',' ',stripped)
    no_elongated = re.sub(r'(.)\1+', r'\1\1', letters_only)
    stopw = stopword.remove(no_elongated)
    stemmed = stemmer.stem(stopw)
    words = [x for x in tok.tokenize(stemmed) if len(x) > 2]
    return (" ".join(words)).strip()

In [8]:
print("Cleaning and parsing the comments...\n")
clean_tweet_texts = []
for i in range(0,2952):
    if((i+1)%100 == 0):
        print("Comments %d of %d has been processed" % (i+1, 100))
    clean_tweet_texts.append(tweet_cleaner(ds['text'][i]))
    
clean_df = pd.DataFrame(clean_tweet_texts, columns=['text'])
clean_df['waktu'] = ds.waktu
clean_df['jumlah kata'] = clean_df.text.apply(lambda x: len(x.split()))

clean_df['waktu'] =  pd.to_datetime(clean_df['waktu'], format='%a %b %d %H:%M:%S %z %Y')
clean_df['jam'] = clean_df.waktu.dt.hour
clean_df['jam'] = pd.cut(clean_df.jam, bins=[0,8,16,24], labels=['pagi','siang','malam'])


clean_df['label'] = ds.label

Cleaning and parsing the comments...

Comments 100 of 100 has been processed
Comments 200 of 100 has been processed
Comments 300 of 100 has been processed
Comments 400 of 100 has been processed
Comments 500 of 100 has been processed
Comments 600 of 100 has been processed
Comments 700 of 100 has been processed
Comments 800 of 100 has been processed
Comments 900 of 100 has been processed
Comments 1000 of 100 has been processed
Comments 1100 of 100 has been processed
Comments 1200 of 100 has been processed
Comments 1300 of 100 has been processed
Comments 1400 of 100 has been processed
Comments 1500 of 100 has been processed
Comments 1600 of 100 has been processed
Comments 1700 of 100 has been processed
Comments 1800 of 100 has been processed
Comments 1900 of 100 has been processed
Comments 2000 of 100 has been processed
Comments 2100 of 100 has been processed
Comments 2200 of 100 has been processed
Comments 2300 of 100 has been processed
Comments 2400 of 100 has been processed
Comments 25

In [9]:
clean_df

,text,waktu,jumlah kata,jam,label
0,sch kata nih tahun ajar baru sekolah offline k...,2021-04-23 12:38:07+00:00,23,siang,-1.0
1,jujur aja gamau kuliah offline kelas paling en...,2021-04-23 11:40:07+00:00,22,siang,-1.0
2,jangan terlalu semangat masuk kelas pagi pagi ...,2021-04-23 11:00:30+00:00,13,siang,-1.0
3,iya asli abis kelas tuh kyk cape banget wkwkwk...,2021-04-23 09:16:45+00:00,18,siang,-1.0
4,sch kalo kelas butuh tambah les offline luar s...,2021-04-23 08:19:58+00:00,22,pagi,-1.0
...,...,...,...,...,...
2947,kemarin sih kepala dinas provinsi aku bilang p...,2021-04-18 09:33:59+00:00,21,siang,1.0
2948,gemush warna anyway enak banget udah sekolah o...,2021-04-23 02:28:37+00:00,8,pagi,1.0
2949,ken sekolah offline,2021-04-21 00:52:50+00:00,3,NaN,1.0
2950,gapapa sampe jumat aja aku abin beri semangat ...,2021-04-20 13:05:15+00:00,18,siang,1.0


In [10]:
#clean_df.fillna(0)
clean_df['jam'].fillna('pagi', inplace =True)
clean_df['label'].fillna('0', inplace =True)

In [11]:
clean_df.drop_duplicates(subset='text', keep="last")

,text,waktu,jumlah kata,jam,label
2,jangan terlalu semangat masuk kelas pagi pagi ...,2021-04-23 11:00:30+00:00,13,siang,-1.0
3,iya asli abis kelas tuh kyk cape banget wkwkwk...,2021-04-23 09:16:45+00:00,18,siang,-1.0
5,punten nanya emang dulu pas kelas offline teru...,2021-04-23 07:49:12+00:00,15,pagi,-1.0
6,kampus max kok cuma kelas asdos hitung max sks...,2021-04-23 04:50:10+00:00,28,pagi,-1.0
7,males bgt mau offline nanggung loh bentar naik...,2021-04-23 04:19:52+00:00,11,pagi,-1.0
...,...,...,...,...,...
2947,kemarin sih kepala dinas provinsi aku bilang p...,2021-04-18 09:33:59+00:00,21,siang,1.0
2948,gemush warna anyway enak banget udah sekolah o...,2021-04-23 02:28:37+00:00,8,pagi,1.0
2949,ken sekolah offline,2021-04-21 00:52:50+00:00,3,pagi,1.0
2950,gapapa sampe jumat aja aku abin beri semangat ...,2021-04-20 13:05:15+00:00,18,siang,1.0


In [12]:
clean_df.to_csv('KA\data\Data1 Clean.csv',encoding='utf-8')